# Summarising DataFrames

In [1]:
import pandas as pd
import seaborn as sns

In [7]:
passengers = sns.load_dataset("flights")
passengers.head()

,year,month,passengers
0,1949,Jan,112
1,1949,Feb,118
2,1949,Mar,132
3,1949,Apr,129
4,1949,May,121


In [8]:
passengers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   year        144 non-null    int64   
 1   month       144 non-null    category
 2   passengers  144 non-null    int64   
dtypes: category(1), int64(2)
memory usage: 2.9 KB


In [12]:
# method summary stats
print(passengers["passengers"].mean())
print(passengers["passengers"].median())
print(passengers["passengers"].max())
print(passengers["passengers"].min())

280.2986111111111
265.5
622
104


In [13]:
# cumulative methods such as cumsum return a column of values
passengers["cumulative_passengers"] = passengers["passengers"].cumsum()
print(passengers.head())

   year month  passengers  cumulative_passengers
0  1949   Jan         112                    112
1  1949   Feb         118                    230
2  1949   Mar         132                    362
3  1949   Apr         129                    491
4  1949   May         121                    612


In [14]:
# setting new records with cummax
passengers["cumulative_maximum"] = passengers["passengers"].cummax()
passengers.head()

,year,month,passengers,cumulative_passengers,cumulative_maximum
0,1949,Jan,112,112,112
1,1949,Feb,118,230,118
2,1949,Mar,132,362,132
3,1949,Apr,129,491,132
4,1949,May,121,612,132


## `.agg()`method

If you need to apply a custom function to a column (or more than one column), then agg
is a good bet.

In [16]:
def get_upper_fence(col):
    q3 = col.quantile(0.75)
    iqr = q3 - col.quantile(0.25)
    return q3 + 1.5 * iqr

up_fence = passengers["passengers"].agg(get_upper_fence)
passengers[passengers["passengers"] > up_fence]
# no upper outliers exist


,year,month,passengers,cumulative_passengers,cumulative_maximum


In [28]:
def get_lower_fence(col):
    q1 = col.quantile(0.25)
    iqr = col.quantile(0.75) - q1
    return q1 - 1.5 * iqr

passengers["passengers"].agg([get_upper_fence, get_lower_fence])
# agg allows multiple defined funcs to be passes in a list.

get_upper_fence    631.25
get_lower_fence    -90.75
Name: passengers, dtype: float64

In [30]:
# you can also pass multiple columns for summary in a list
passengers[["year", "passengers"]].agg([get_upper_fence, get_lower_fence])

,year,passengers
get_upper_fence,1965.5,631.25
get_lower_fence,1943.5,-90.75
